In [29]:
import xgboost as xgb
from read_files import X,y
from sklearn.model_selection import train_test_split
from classifier_methods import *
from sklearn.metrics import accuracy_score

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [9]:
X_train_features, *settings = get_features_train(X_train)

Generating Train Features
Generating Train Features
Processing features: pos_counts


100%|██████████| 315/315 [00:54<00:00,  5.78it/s]


Individual Predict Processing for features: pos_counts, is complete!
Processing features: author_style_counts


100%|██████████| 315/315 [00:16<00:00, 18.89it/s]


Individual Predict Processing for features: author_style_counts, is complete!
Processing features: lix_score


...

Emoji TF_IDF Processing for features: tf_idf, is complete!
Generating IDF Vector
Processing features: tf_idf


100%|██████████| 315/315 [00:06<00:00, 46.22it/s]


Profanity TF_IDF Processing for features: tf_idf, is complete!
Generating IDF Vector
Processing features: tf_idf


100%|██████████| 315/315 [00:17<00:00, 18.53it/s]


Words TF_IDF Processing for features: tf_idf, is complete!
Emoji Explained Variance:  0.42733522772932125
Profanity Explained Variance:  0.482693069316176
Word Explained Variance:  0.4771719434857197


In [12]:
X_test_features = get_features_test(X_test, *settings)

Generating Test Features
Processing features: pos_counts


100%|██████████| 105/105 [00:18<00:00,  5.56it/s]


Individual Predict Processing for features: pos_counts, is complete!
Processing features: author_style_counts


100%|██████████| 105/105 [00:05<00:00, 20.86it/s]


Individual Predict Processing for features: author_style_counts, is complete!
Processing features: lix_score


...

Individual Predict Processing for features: tf_idf, is complete!
Processing features: tf_idf


100%|██████████| 105/105 [00:02<00:00, 45.10it/s]


Individual Predict Processing for features: tf_idf, is complete!
Processing features: tf_idf


100%|██████████| 105/105 [00:05<00:00, 18.75it/s]

Words TF_IDF Processing for features: tf_idf, is complete!


In [18]:
y_train[y_train == 'I'] = 1
y_train[y_train == 'NI'] = 0

y_test[y_test == 'I'] = 1
y_test[y_test == 'NI'] = 0

In [17]:
dtrain = xgb.DMatrix(X_train_features, label=y_train)

In [53]:
param = {'max_depth': 4, 'eta': 1, 'objective': 'binary:hinge', 'eval_metric':'error'}
num_round = 10
bst = xgb.train(param, dtrain, num_round)

In [54]:
dtest = xgb.DMatrix(X_test_features)
ypred = bst.predict(dtest)
np.sum(y_test.astype(int) == ypred)/105


0.9047619047619048

TypeError: only size-1 arrays can be converted to Python scalars

In [62]:
reg = xgb.XGBClassifier(tree_method="hist")
# Fit the model using predictor X and response y.
reg.fit(X_train_features, y_train.astype(int))


[15:27:55] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='hist', validate_parameters=1, verbosity=None)

In [64]:
from sklearn.model_selection import GridSearchCV

np.random.seed(1)

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions != test_labels)
    accuracy = (predictions==test_labels).sum()/len(test_labels)
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    return accuracy


clf_rfc = xgb.XGBClassifier(eval_metric = 'error')

clf_rfc.fit(X_train_features, y_train.astype(int))
base_accuracy = evaluate(clf_rfc, X_test_features, y_test.astype(int))

clf_rfc = xgb.XGBClassifier(eval_metric = 'error', use_label_encoder = False)

param_grid = {
    'n_estimators': [100, 150, 180, 200],
    'booster': ['gbtree', 'gblinear', 'dart'],
    'tree_method': ['hist', 'auto', 'exact'],
    'eta': np.arange(0,5,0.1), 
    'max_depth': np.arange(3,15,2),
    'min_child_weight': np.arange(1,100,10),
    'max_delta_step' : np.arange(1,10,1),
    'subsample' : np.arange(0,1,0.2),
}

grid_search = GridSearchCV(estimator = clf_rfc, param_grid = param_grid, cv = 4, n_jobs = -1, verbose = 1)
grid_search.fit(X_train_features, y_train.astype(int))
print(grid_search.best_params_)

best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test_features, y_test.astype(int))

print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))




/Users/yananikolova/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Model Performance
Average Error: 0.0857 degrees.
Accuracy = 0.91%.
Fitting 4 folds for each of 4860000 candidates, totalling 19440000 fits


/Users/yananikolova/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/yananikolova/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/yananikolova/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/yananikolova/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from p

KeyboardInterrupt: 

In [ ]:
evaluate()